In [30]:
import numpy as np
import pandas as pd

df = pd.read_csv('time.csv', sep = '\t', names = ["inter_arrival","service_time"])
df

,inter_arrival,service_time
0,0.167418,0.273090
1,4.069272,0.003407
2,0.362473,0.088248
3,0.037700,0.064942
4,1.260989,0.619072
...,...,...
2995,0.534943,0.235275
2996,0.074017,0.890757
2997,0.248282,0.798065
2998,0.016911,0.065893


In [33]:
class Simulation:
    def __init__(self, df): 
        self.clock=0.0                      #simulation clock
        self.num_arrivals=0                 #total number of arrivals
        self.df = df
        self.t_arrival=self.gen_int_arr()   #time of next arrival
        self.t_departure1=float('inf')      #departure time from server 1
        self.dep_sum1=0                     #Sum of service times by teller 1
        self.state_T1=0                     #current state of server1 (binary)
        self.total_wait_time=0.0            #total wait time
        self.num_in_q=0                     #current number in queue
        self.number_in_queue=0              #customers who had to wait in line(counter)
        self.num_of_departures1=0           #number of customers served by teller 1 

    def time_adv(self):                                                       
        t_next_event=min(self.t_arrival,self.t_departure1)  
        self.total_wait_time += (self.num_in_q*(t_next_event-self.clock))
        self.clock=t_next_event
                
        if self.t_arrival<self.t_departure1:
            self.arrival()
        elif self.t_departure1<self.t_arrival:
            self.server1()
            
    def arrival(self):              
        self.num_arrivals += 1
        if self.num_in_q == 0:                                
            if self.state_T1==1:
                self.num_in_q+=1
                self.number_in_queue+=1
                self.t_arrival=self.clock+self.gen_int_arr()
            else:
                self.state_T1=1
                self.dep1= self.gen_service_time_server1()
                self.dep_sum1 += self.dep1
                self.t_departure1=self.clock + self.dep1
                self.t_arrival=self.clock+self.gen_int_arr()
        else:                         
                self.t_arrival=self.clock+self.gen_int_arr()
                self.num_in_q+=1 
                self.number_in_queue+=1
                

    def server1(self):              
        self.num_of_departures1 += 1
        if self.num_in_q>0:
            self.dep1= self.gen_service_time_server1()
            self.dep_sum1 += self.dep1
            self.t_departure1=self.clock + self.dep1
            self.num_in_q-=1
        else:
            self.t_departure1=float('inf') 
            self.state_T1=0                  

            
    def gen_int_arr(self):                                            
        return float(df.sample(1)['inter_arrival'])
    
    def gen_service_time_server1(self):                               
        return float(df.sample(1)['service_time'])
    

In [48]:
sum = 0
for i in range(10):
    s=Simulation(df)
    while s.num_arrivals <= 3000 :
        s.time_adv()
    sum+= s.total_wait_time/s.num_arrivals    
    print(sum)
total_time_10_simulations = sum / 10

5.128906153120152
11.469170395290568
17.519136058248485
26.1403706347866
34.6090736485402
44.66240954054871
51.7596639642195
57.6420108227905
62.929273315563876
67.65871741886444


In [49]:
# APPROXIMATIONS USING FORMULAS.

lambd = 1 / df['inter_arrival'].mean() 
mu = 1 / df['service_time'].mean()
rho = lambd / mu

E_TQ = rho / (mu - lambd)
print('Waiting time in Queue in a M/M/1 system is E[T_Q] = {} seconds.'.format(E_TQ))
t = 1 / mu

cs =  df['service_time'].std() / df['service_time'].mean()
ca = df['inter_arrival'].std() / df['inter_arrival'].mean()
E_WQ = (rho/(1-rho)) * ((cs**2 + ca**2)/2) * t
print('Waiting time in Queue in a G/G/1 system is W[T_Q] = {} seconds.'.format(E_WQ))

print('Waiting time in Queue from the Simulation G/G/1 : {} seconds.'.format(total_time_10_simulations))

Waiting time in Queue in a M/M/1 system is E[T_Q] = 1.0117026459940368 seconds.
Waiting time in Queue in a G/G/1 system is W[T_Q] = 6.550551347672369 seconds.
Waiting time in Queue from the Simulation G/G/1 : 6.7658717418864445
